# Colab-RIFE

Original repo: [hzwer/arXiv2020-RIFE](https://github.com/hzwer/arXiv2020-RIFE)

In [ ]:
!nvidia-smi

In [ ]:
#@title Setup
!git clone https://github.com/hzwer/Arxiv2020-RIFE
!mkdir /content/Arxiv2020-RIFE/input
!mkdir /content/Arxiv2020-RIFE/output
!mkdir /content/Arxiv2020-RIFE/train_log

!sudo apt install ffmpeg

!pip install gdown
%cd /content/Arxiv2020-RIFE/train_log
!gdown --id 1c1R7iF-ypN6USo-D2YH_ORtaH3tukSlo
!7z e RIFE_trained_model.zip

In [ ]:
#@title test.py (Original by [_Jacob](https://github.com/buildist) with some modifications)
# Original: https://pastebin.com/raw/XzZgeqbs
%%writefile /content/Arxiv2020-RIFE/test.py
import cv2
import os
import numpy as np
import shutil
import torch
import torchvision
from torchvision import transforms
from torch.nn import functional as F
from PIL import Image
from model.RIFE import Model
from glob import glob
from imageio import imread, imsave
from torch.autograd import Variable
from tqdm import tqdm
video_paths = ['/content/Arxiv2020-RIFE/']
video_paths_ = glob(video_paths[0] + '/**/*.mp4', recursive=True)


for path in video_paths_:
    name = os.path.basename(path)
    print(name)
    if os.path.isfile(path):
        output_path = path.replace('input', 'tmp')
        os.makedirs(output_path, exist_ok = True)
        video_paths.append(output_path)
        if not os.path.isfile('{:s}/00000001.png'.format(output_path)):
            os.system('ffmpeg -i {:s} {:s}/%08d.png -hide_banner'.format(path, output_path));

RIFE_model = Model()
RIFE_model.load_model('./train_log')
RIFE_model.eval()
RIFE_model.device()

for path in video_paths_:
    name = os.path.basename(path)
    length = len(glob(path + '/*.png'))
    sr_output_path = path.replace(name, name+'_sr_out')
    os.makedirs(sr_output_path, exist_ok = True)
    interp_output_path = path.replace(name, name+'_interp_out')
    os.makedirs(interp_output_path, exist_ok = True)
    output_path = path.replace('tmp', 'output')
    if os.path.isfile(output_path):
        continue

    with torch.no_grad():
        if not os.path.isfile('{:s}/00000001.png'.format(interp_output_path)):
            output_frame_number = 1
            for input_frame_number in tqdm(range(1, length)):
                frame_0_path = 'tmp/'+'{:s}/{:08d}.png'.format(name, input_frame_number)
                frame_1_path = 'tmp/'+'{:s}/{:08d}.png'.format(name, input_frame_number + 1)

                frame_0 = cv2.imread(frame_0_path)
                frame_1 = cv2.imread(frame_1_path)

                h, w, _ = frame_0.shape
                ph = h // 32 * 32+32
                pw = w // 32 * 32+32
                padding = (0, pw - w, 0, ph - h)
                frame_0 = torch.tensor(frame_0.transpose(2, 0, 1)).cuda() / 255.
                frame_1 = torch.tensor(frame_1.transpose(2, 0, 1)).cuda() / 255.
                imgs = F.pad(torch.cat((frame_0, frame_1), 0).float(), padding)
                res = RIFE_model.inference(imgs.unsqueeze(0)) * 255

                shutil.copyfile(frame_0_path, '{:s}/{:08d}.png'.format(interp_output_path, output_frame_number))
                output_frame_number += 1
                cv2.imwrite('{:s}/{:08d}.png'.format(interp_output_path, output_frame_number), res[0].cpu().numpy().transpose(1, 2, 0)[:h, :w])
                output_frame_number += 1

                if output_frame_number == length*2 - 1:
                    shutil.copyfile(frame_1_path, '{:s}/{:08d}.png'.format(interp_output_path, output_frame_number))
                    output_frame_number += 1
    os.makedirs(output_path.replace(os.path.basename(output_path), ''), exist_ok = True)
    source_path = path.replace('tmp', 'input')
    #os.system('ffmpeg -y -f image2 -r 30 -i {:s}/%08d.png -i {:s} -c copy -map 0:0 -map 1:1? -crf 3 -c:v mjpeg -q:v 3 {:s} -hide_banner'.format(interp_output_path, source_path, output_path));
    os.system('ffmpeg -y -f image2 -r 47.952 -i {:s}/%08d.png -i {:s} -c copy -map 0:0 -map 1:1? -crf 3 -c:v libx264 -q:v 8 {:s} -hide_banner'.format(interp_output_path, source_path, output_path));

Place ```.mp4``` videos in ```/input``` and result will be in ```/output```. Run it twice in a row.

If you want to change framerate or ffmpeg settings, do that in ```test.py```. 

Don't forget to input 2x FPS in the ffmpeg command.

In [ ]:
%cd /content/Arxiv2020-RIFE
!python test.py

In [ ]:
#@title Delete all files
%cd /content/
!sudo rm -rf /content/Arxiv2020-RIFE/input
!sudo rm -rf /content/Arxiv2020-RIFE/output
!sudo rm -rf /content/Arxiv2020-RIFE/tmp

!mkdir /content/Arxiv2020-RIFE/input
!mkdir /content/Arxiv2020-RIFE/output
!mkdir /content/Arxiv2020-RIFE/tmp